In [45]:
if (Sys.info()[1] == "Linux"){
  j <- "/home/j"
  h <- paste0("/home/",Sys.info()[6]) # what is this 6?
  package_lib <- paste0(j,'/temp/stearns7/packages_cl') # Library for packages on cluster. Ensures that none of this code is dependent on the machine where the user runs the code.
  repo <- '/share/code/geospatial/stearns7/eco_niche'
}else{
  j <- "J:"
  h <- "H:"
  package_lib <- paste0(j,'/temp/stearns7/packages') #library for packages locally
}
library('parallel')
slots = 25
cores_to_use = ifelse(grepl('Intel', system("cat /proc/cpuinfo | grep \'name\'| uniq", inter = T)), floor(slots * .86), floor(slots*.64))



Attaching package: ‘parallel’

The following objects are masked from ‘package:snow’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, makeCluster, parApply,
    parCapply, parLapply, parRapply, parSapply, splitIndices,
    stopCluster



In [3]:
## Set data location
data_loc <- (paste0(j, '/temp/stearns7/schisto/data/eco_niche_data'))

## Load libraries
setwd(repo)

# Library for packages. Ensures that none of this code is dependent on the machine where the user runs the code.
.libPaths(package_lib)# Ensures packages look for dependencies here when called with library().

# Load functions files
source(paste0(repo, '/econiche_central/functions.R'))                   
source(paste0(repo, '/econiche_central/econiche_qsub.R'))  
source(paste0(repo, '/econiche_central/check_loc_results.R'))

In [4]:
package_list <- c('car', 'MASS', 'ggplot2', 'rgeos', 'data.table','raster','rgdal', 'seegSDM','sp')
for(package in package_list) {
  library(package, lib.loc = package_lib, character.only=TRUE)
}

rgeos version: 0.3-22, (SVN revision 544)
 GEOS runtime version: 3.3.2-CAPI-1.7.2 
 Linking to sp version: 1.2-4 
 Polygon checking: TRUE 

Loading required package: sp

Attaching package: ‘raster’

The following object is masked from ‘package:data.table’:

    shift

The following objects are masked from ‘package:MASS’:

    area, select

rgdal: version: 1.2-4, (SVN revision 643)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.7.3, released 2010/11/10
 Path to GDAL shared files: /usr/share/gdal
 GDAL does not use iconv for recoding strings.
 Loaded PROJ.4 runtime: Rel. 4.7.1, 23 September 2009, [PJ_VERSION: 470]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-3 
Warning message:
“package ‘seegSDM’ was built under R version 3.3.2”Loading required package: snowfall
Loading required package: snow


In [5]:
## Create run date in correct format - calls make_time_stamp function from 'functions' - copied from Nick Graetz's in 'prep_functions' for MBG code
time_stamp <- TRUE 
run_date <- make_time_stamp(time_stamp)

# Set output path
outpath <- (paste0(data_loc, '/output'))

In [6]:
# Load covariate raster brick here (created ahead of time)
covs <- brick(paste0(data_loc, "/covariates/schisto_covs.grd"))

In [8]:
occ <- read.csv(file = (paste0(data_loc, '/man_fin.csv')))

In [27]:
# Generate pseudo-absence data according to the aridity surface and suppress weighting (prob=FALSE) so as to not weight by aridity pixel values
aridity <- raster(paste0(data_loc, "/covariates/aridity_annual.tif"))
print('Loading grid for background point generation')

bg <- bgSample(aridity, # Weighting grid - population in this case, custom function defined in github 
               n = 1600, # Background data points desired
               prob = FALSE, # Set to FALSE so doesn't weight by raster specified above
               replace = TRUE,
spatial = FALSE)

colnames(bg) <- c('long', 'lat') 
bg = data.frame(bg)

[1] "Loading grid for background point generation"


In [28]:
# Add an outbreak id to this
bg$outbreak_id <- 0

In [29]:
#this seems convoluted
dat <- rbind(cbind(PA = rep(1, nrow(occ)),
                   occ[, c('long', 'lat', 'outbreak_id')]),
             cbind(PA = rep(0, nrow(bg)),bg))

In [30]:
#need to drop 'outbreak_id'
dat <- dat[,c(1:3)]


In [31]:
dat_covs <- extract(covs, dat[, 2:3])

In [32]:
dat_all <- cbind(dat, dat_covs)

In [33]:
dat_all <- na.omit(dat_all)

In [46]:
#create multiple versions of the dataset via sample
data_sample = lapply(1:25, function(x) subsample(dat_all, 800, minimum= c(30,30)))

In [47]:
#Run the brts
models <- mclapply(data_sample, function(x) runBRT(x,
          gbm.x = 4:ncol(x),
          gbm.y = 1,
          pred.raster = covs, #brick
          gbm.coords = 2:3,
          wt = function(PA) ifelse(PA == 1, 1, sum(PA) / sum(1 - PA))),mc.cores = cores_to_use )

Warning message:
“closing unused connection 4 (/tmp/RtmpijLiOd/raster///r_tmp_2017-01-14_091231_47975_97588.gri)”

In [53]:
#get model stats
model_stats <- suppressWarnings(lapply(models, function(x) getStats(x)))

In [54]:
#get all the prediction results
preds <- brick(lapply(models, '[[', 4)) #4th component likely the prediction raster layer
# summarise the predictions in parallel
preds_sry <- combinePreds(preds)

In [55]:
model_stats = rblindlist(model_stats)

ERROR: Error in eval(expr, envir, enclos): could not find function "rblindlist"


In [ ]:
head(model_stats)